In [ ]:
model_path = "liuhaotian/llava-v1.6-vicuna-7b" # @param {"type":"string","placeholder":"\"liuhaotian/llava-v1.6-vicuna-7b\""}
%cd /content

import os, sys

# Clone or update LLaVA repo
if not os.path.exists("/content/LLaVA"):
    !git clone https://github.com/xjdeng/LLaVA
else:
    !git -C /content/LLaVA pull --ff-only

%cd /content/LLaVA

# Clone or update Llava-colab helper repo
if not os.path.exists("/content/LLaVA/Llava-colab"):
    !git clone https://github.com/Besuf/Llava-colab
else:
    !git -C /content/LLaVA/Llava-colab pull --ff-only

# Install a CUDA-compatible PyTorch for Colab
try:
    import torch
    has_cuda = torch.cuda.is_available()
except Exception:
    has_cuda = False

if has_cuda:
    # CUDA 12.1 wheels for recent Colab runtimes
    !pip install -q --upgrade "torch==2.4.0" "torchvision==0.19.0" "torchaudio==2.4.0" --index-url https://download.pytorch.org/whl/cu121
else:
    !pip install -q --upgrade "torch==2.4.0" "torchvision==0.19.0" "torchaudio==2.4.0"

# Core Python deps (pin versions with prebuilt wheels to avoid build failures)
!pip install -q --upgrade "transformers==4.36.2" "tokenizers==0.15.2" "accelerate>=0.23.0" "bitsandbytes>=0.43.1" "gradio>=3.45.0" "einops" "sentencepiece" "pillow"

# Install LLaVA package but skip its strict dependency pins (avoids torch==2.0.1)
!pip install -q -e . --no-deps

# Patch LLaVA to avoid duplicate 'llava' AutoConfig registration conflicts with Transformers
# This makes the registration idempotent if Transformers already provides a 'llava' entry
!python - <<'PY'
import os, re
p = 'llava/model/language_model/llava_llama.py'
if os.path.exists(p):
    s = open(p, 'r', encoding='utf-8').read()
    new = re.sub(r'AutoConfig\.register\("llava",\s*LlavaConfig\)',
                 'AutoConfig.register("llava", LlavaConfig, exist_ok=True)', s)
    if new != s:
        open(p, 'w', encoding='utf-8').write(new)
        print('Patched: AutoConfig.register("llava", LlavaConfig, exist_ok=True)')
    else:
        print('No patch needed for AutoConfig.register; pattern not found or already patched')
else:
    print('Patch skipped: file not found', p)

In [ ]:
%cd Llava-colab
!git pull
%cd ..
!cp Llava-colab/backend.py .

In [ ]:
!python backend.py $model_path